# PyTorch

In [1]:
import torch

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchvision import datasets, transforms

In [2]:
seed = 1

lr = 0.001
momentum = 0.5

batch_size = 64
test_batch_size = 64

epochs = 5

no_cuda = False
log_interval = 100

## Model

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 20, 5, 1) # gray scale은 안되어서 (1,20,5,1) -> (3,20,5,1)  ( channel 수를 늘임 )
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    # conv1 -> ReLU -> max pooling -> conv2 -> ReLU -> max pooling -> (view) FC1 -> ReLU -> FC2 -> Softmax
    def forward(self, x):
        x = F.relu(self.conv1(x))  
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

## Preprocess

In [10]:
import os
os.listdir('dataset/mnist_png/training/')

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

- train & test 경로 지정

In [4]:
train_dir = 'dataset/mnist_png/training/'
test_dir = 'dataset/mnist_png/testing/'

grayscale은 안 되는 이유

https://github.com/pytorch/vision/blob/master/torchvision/datasets/folder.py#L157

In [14]:
torch.manual_seed(seed)

use_cuda = not no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

#############################################################

# dataset 
train_dataset = datasets.ImageFolder(root=train_dir,
                           transform=transforms.Compose([
                               transforms.ToTensor(),
                               transforms.Normalize((0.1307,), (0.3081,))
                           ]))

test_dataset = datasets.ImageFolder(root=test_dir,
                           transform=transforms.Compose([
                               transforms.ToTensor(),
                               transforms.Normalize((0.1307,), (0.3081,))
                           ]))

#################################################################

# data loader
train_loader = torch.utils.data.DataLoader(train_dataset,
                                         batch_size=batch_size,
                                         shuffle=True)

test_loader = torch.utils.data.DataLoader(test_dataset,
                                         batch_size=test_batch_size
                                         )

## Optimization

In [13]:
model = Net().to(device)
#optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
optimizer = optim.Adam(model.parameters(), lr=lr)

## Training

In [15]:
for epoch in range(1, epochs + 1):
    
    ############# Train Mode ################
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()  # backpropagation 계산하기 전에 0으로 기울기 계산
        output = model(data)
        loss = F.nll_loss(output, target)  # https://pytorch.org/docs/stable/nn.html#nll-loss
        loss.backward()  # 계산한 기울기를 
        optimizer.step()  

        # 100번째 마다 log 찍어내게끔!
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch,                                                       # 1) epoch
                batch_idx * len(data),                                # 2) n번째 dat (0, 6400, 12800...)
                len(train_loader.dataset),                          # 3)  총 data  ( 60,000 )
                100. * batch_idx / len(train_loader),          # 4) 몇 % 진행되었는지
                loss.item()))                                               # 5) train loss
    
    ############# Test Mode ################
    model.eval()  # batch norm이나 dropout 등을 train mode 변환
    test_loss = 0
    correct = 0
    with torch.no_grad():  # autograd engine, 즉 backpropagatin이나 gradient 계산 등을 꺼서 memory usage를 줄이고 속도를 높임
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()  # pred와 target과 같은지 확인

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.328665
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.228394
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.067597
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.060442
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.135482
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.033144
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.009140
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.168844
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.051005


KeyboardInterrupt: 